In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd

In [2]:
# file path to csv
TRAIN_CSV_PATH = 'data/train_cleaned.csv'
TEST_CSV_PATH = 'data/test_cleaned.csv'

# Read in the csv file
train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)

In [3]:
# Preview the train data
train_df.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# Preview th test data (first five rows)
test_df.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [5]:
train_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [6]:
NUMERICAL_COLUMNS = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_weeK']
CATEGORICAL_COLUMNS = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
LABEL = ['income'] 

In [7]:
train_df.income = train_df.income.str.strip()
test_df.income = test_df.income.str.strip()

In [8]:
# Encode training the income which is the label to be O and 1 for binary classification
train_df['income'] = np.where(train_df['income'] == '<=50K',0, 1)

# Encode test data
test_df['income'] = np.where(test_df['income'] == '<=50K',0, 1)

In [9]:
train_df.income.value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [10]:
test_df.income.value_counts()

0    12435
1     3846
Name: income, dtype: int64

In [11]:
#train_df.sex.value_counts()
train_df.sex = np.where(train_df.sex == 'Male', 0, 1)
test_df.sex = np.where(test_df.sex == 'Male', 0, 1)

In [12]:
train_df.sex.value_counts()

0    21790
1    10771
Name: sex, dtype: int64

In [13]:
test_df.sex.value_counts()

0    10860
1     5421
Name: sex, dtype: int64

In [14]:
train_df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,0,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,0,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,0,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,0,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,1,0,0,40,Cuba,0


In [16]:
BUCKET_FEATURE = 'age'
NUM_OF_BUCKET = 4
CUSTOM_LABEL = ['young', 'adult', 'middle_age', 'old']

# features to compute vocabulary on
VOCABULARY_FEATURES = ['education', 'relationship', 'marital_status', 'occupation']

transformation_utils = 'transformation/transform_utils.py'

In [17]:
%%writefile {transformation_utils}
import numpy as np
import pandas as pd

def bucketize_with_labels(data, column_name, num_buckets, custom_labels):
    """
     Bcketize a numerical feature with custom labels.

    Args:
        data (pd.DataFrame): The input DataFrame.
        column_name (str): Name of the column to be bucketized.
        num_buckets (int): Number of desired buckets.
        custom_labels (list of str): Custom labels for each bucket.

    Returns:
        pd.Series: A new series with bucket labels.
    """
    # Extract the specified column
    column_values = data[column_name]

    # Use pd.qcut to create quantile-based buckets
    bucketized_series = pd.cut(column_values, bins=num_buckets, labels=custom_labels)

    return bucketized_series

def compute_and_apply_vocabulary(df, column):
    """
    Custom function to compute vocabulary and map words to integer indices.

    Args:
        df (DataFrame): pandas data frame.
        column (str): name of the desired feature.
        

    Returns:
        dict: A dictionary mapping unique words to integer indices.
        series: A Panda series of integers
    """
    # Step 1: Extract all words from the documents
    list_of_words = list(df[column])

    # Step 2: Create a vocabulary dictionary
    vocabulary = {}
    for idx, word in enumerate(set(list_of_words)):
        vocabulary[word] = idx

    # Step 3: Replace words in the documents with their corresponding indices
    
    new_list = df[column].replace(vocabulary)

    return new_list, vocabulary


def min_max_scaler(df, column_name):
    """
    Scales a specific column to the range [0, 1].

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): Name of the column to be scaled.

    Returns:
        panda.core.series.Series: The DataFrame with the scaled column.
    """
    # Extract the specified column
    column_values = df[column_name]

    # Calculate the minimum and maximum values
    min_val = column_values.min()
    max_val = column_values.max()

    # Scale the column to [0, 1]
    scaled_column = (column_values - min_val) / (max_val - min_val)

    return scaled_column

Overwriting transformation/transform_utils.py


In [31]:
from transformation.transform_utils import bucketize_with_labels

age_mapping = {'young':0, 'adult':1, 'middle_age':2, 'old':3}
#j  = bucketize_with_labels(train_df, BUCKET_FEATURE, NUM_OF_BUCKET, CUSTOM_LABEL).replace(age_mapping)
train_df['age'] = bucketize_with_labels(train_df, BUCKET_FEATURE, NUM_OF_BUCKET, CUSTOM_LABEL).replace(age_mapping)
test_df['age'] = bucketize_with_labels(test_df, BUCKET_FEATURE, NUM_OF_BUCKET, CUSTOM_LABEL).replace(age_mapping)

In [1]:
train_df['marital_status'] = train_df['marital_status'].replace({'Married-civ-spouse': 'Married', 'Never-married': 'Single',  
                                                        'Separated':'Divorced', 'Married-spouse-absent' : 'Divorced', 
                                                         'Divorced':'Divorced', 
                                                         'Married-AF-spouse' :'Divorced', 'Widowed':'Widowed'})

# Test
test_df['marital_status'] = test_df['marital_status'].replace({'Married-civ-spouse': 'Married', 'Never-married': 'Single',  
                                                        'Separated':'Divorced', 'Married-spouse-absent' : 'Divorced', 
                                                         'Divorced':'Divorced', 
                                                         'Married-AF-spouse' :'Divorced', 'Widowed':'Widowed'})

NameError: name 'train_df' is not defined

In [23]:
train_df.marital_status.value_counts()

Married     14976
Single      10683
Divorced     5909
Widowed       993
Name: marital_status, dtype: int64

In [24]:
test_df.marital_status.value_counts()

Married     7403
Single      5434
Divorced    2919
Widowed      525
Name: marital_status, dtype: int64

In [25]:
# Create Vocabulary for the feature
#occupation_ordered = train_df.groupby(['occupation'])['income'].count().sort_values().index
#occupation_ordered = {k:i for i,k in enumerate(occupation_ordered)}
#occupation_ordered
#train_df['occupation'] = train_df['occupation'].map(occupation_ordered)

{'Armed-Forces': 1,
 'Priv-house-serv': 2,
 'Protective-serv': 3,
 'Tech-support': 4,
 'Farming-fishing': 5,
 'Handlers-cleaners': 6,
 'Transport-moving': 7,
 '?': 8,
 'Machine-op-inspct': 9,
 'Other-service': 10,
 'Sales': 11,
 'Adm-clerical': 12,
 'Exec-managerial': 13,
 'Craft-repair': 14,
 'Prof-specialty': 15}

In [25]:
from transformation.transform_utils import compute_and_apply_vocabulary

for feature in VOCABULARY_FEATURES:
    train_df[feature], vocab_dict = compute_and_apply_vocabulary(train_df, feature)
    print(f"Training set vocabulary dictionary for {feature}")
    print(vocab_dict)
    print("\n")
    test_df[feature], vocab_ = compute_and_apply_vocabulary(test_df, feature)
    print(f"Test set vocabulary dictionary for {feature}")
    print(vocab_dict)
    print("\n")

Training set vocabulary dictionary for education
{'Assoc-voc': 0, 'Some-college': 1, '9th': 2, '7th-8th': 3, '1st-4th': 4, 'Doctorate': 5, 'Masters': 6, 'HS-grad': 7, 'Prof-school': 8, '5th-6th': 9, '11th': 10, '12th': 11, 'Preschool': 12, 'Assoc-acdm': 13, 'Bachelors': 14, '10th': 15}
Test set vocabulary dictionary for education
{'Assoc-voc': 0, 'Some-college': 1, '9th': 2, '7th-8th': 3, '1st-4th': 4, 'Doctorate': 5, 'Masters': 6, 'HS-grad': 7, 'Prof-school': 8, '5th-6th': 9, '11th': 10, '12th': 11, 'Preschool': 12, 'Assoc-acdm': 13, 'Bachelors': 14, '10th': 15}
Training set vocabulary dictionary for relationship
{'Not-in-family': 0, 'Other-relative': 1, 'Husband': 2, 'Wife': 3, 'Unmarried': 4, 'Own-child': 5}
Test set vocabulary dictionary for relationship
{'Not-in-family': 0, 'Other-relative': 1, 'Husband': 2, 'Wife': 3, 'Unmarried': 4, 'Own-child': 5}
Training set vocabulary dictionary for marital_status
{'Divorced': 0, 'Single': 1, 'Widowed': 2, 'Married': 3}
Test set vocabulary d

In [26]:
train_df.head(10)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,14,13,1,8,0,White,0,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,14,13,3,2,2,White,0,0,0,13,United-States,0
2,38,Private,215646,7,9,0,5,0,White,0,0,0,40,United-States,0
3,53,Private,234721,10,7,3,5,2,Black,0,0,0,40,United-States,0
4,28,Private,338409,14,13,3,12,3,Black,1,0,0,40,Cuba,0
5,37,Private,284582,6,14,3,2,3,White,1,0,0,40,United-States,0
6,49,Private,160187,2,5,0,10,0,Black,1,0,0,16,Jamaica,0
7,52,Self-emp-not-inc,209642,7,9,3,2,2,White,0,0,0,45,United-States,1
8,31,Private,45781,6,14,1,12,0,White,1,14084,0,50,United-States,1
9,42,Private,159449,14,13,3,2,2,White,0,5178,0,40,United-States,1


In [27]:
# Features to encode zero and one because a very high percentage of the data is more than the rest
features_to_encode = ['workclass', 'native_country', 'race']

In [28]:
# Transform workclass feature to be 1 if the workclass is Private and 0 if it isn't
train_df['workclass'] = np.where(train_df['workclass'] == 'Private',1,0)

# Transform native_country feature to be 1 if the native country is United-States and 0 if it isn't
train_df['native_country'] = np.where(train_df['native_country'] == 'United-States',1,0)

# Transform race feature to be 1 if the race is White and 0 if it isn't
train_df['race'] = np.where(train_df['race'] == 'White',1,0)

In [29]:
# Test set
# Transform workclass feature to be 1 if the workclass is Private and 0 if it isn't
test_df['workclass'] = np.where(test_df['workclass'] == 'Private',1,0)

# Transform native_country feature to be 1 if the native country is United-States and 0 if it isn't
test_df['native_country'] = np.where(test_df['native_country'] == 'United-States',1,0)

# Transform race feature to be 1 if the race is White and 0 if it isn't
test_df['race'] = np.where(test_df['race'] == 'White',1,0)

In [32]:
train_df.head(10)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,1,0,77516,14,13,1,8,0,1,0,2174,0,40,1,0
1,1,0,83311,14,13,3,2,2,1,0,0,0,13,1,0
2,1,1,215646,7,9,0,5,0,1,0,0,0,40,1,0
3,1,1,234721,10,7,3,5,2,0,0,0,0,40,1,0
4,0,1,338409,14,13,3,12,3,0,1,0,0,40,0,0
5,1,1,284582,6,14,3,2,3,1,1,0,0,40,1,0
6,1,1,160187,2,5,0,10,0,0,1,0,0,16,0,0
7,1,0,209642,7,9,3,2,2,1,0,0,0,45,1,1
8,0,1,45781,6,14,1,12,0,1,1,14084,0,50,1,1
9,1,1,159449,14,13,3,2,2,1,0,5178,0,40,1,1


In [34]:
train_df.age.value_counts()

0    14925
1    12713
2     4455
3      468
Name: age, dtype: int64

In [35]:
test_df.head(10)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,0,1,226802,10,7,1,2,5,0,0,0,0,40,1,0
1,1,1,89814,7,9,3,14,2,1,0,0,0,50,1,0
2,0,0,336951,13,12,3,6,2,1,0,0,0,40,1,1
3,1,1,160323,1,10,3,2,2,0,0,7688,0,40,1,1
4,0,0,103497,1,10,1,13,5,1,1,0,0,30,1,0
5,0,1,198693,15,6,1,10,0,1,0,0,0,30,1,0
6,0,0,227026,7,9,1,13,4,0,0,0,0,40,1,0
7,2,0,104626,8,15,3,12,2,1,0,3103,0,32,1,1
8,0,1,369667,1,10,1,10,4,1,1,0,0,40,1,0
9,2,1,104996,2,4,3,0,2,1,0,0,0,10,1,0
